# Part 4A Queries in PyTables

> Objectives:
> * Query HDF5 files without loading them in-memory
> * How to query normalized and denormalized tables
> * Index columns in tables for accelerating queries

In [1]:
import os
import numpy as np
import pandas as pd
import tables

In [2]:
import os
import shutil
data_dir = "queries"
if os.path.exists(data_dir):
    shutil.rmtree(data_dir)
os.mkdir(data_dir)

In [4]:
!ls -lh compression

total 406M
-rw-r--r-- 1 tomkooij 197613 5.0M Jun 19 14:48 blosc-5-shuffle.h5
-rw-r--r-- 1 tomkooij 197613 7.3M Jun 19 14:48 blosc-blosclz-5-shuffle-denorm.h5
-rw-r--r-- 1 tomkooij 197613 5.0M Jun 19 14:48 blosc-blosclz-5-shuffle.h5
-rw-r--r-- 1 tomkooij 197613 7.8M Jun 19 14:48 blosc-lz4-5-shuffle-denorm.h5
-rw-r--r-- 1 tomkooij 197613 5.4M Jun 19 14:48 blosc-lz4-5-shuffle.h5
-rw-r--r-- 1 tomkooij 197613 6.7M Jun 19 14:48 blosc-lz4hc-5-shuffle-denorm.h5
-rw-r--r-- 1 tomkooij 197613 4.8M Jun 19 14:48 blosc-lz4hc-5-shuffle.h5
-rw-r--r-- 1 tomkooij 197613 156M Jun 19 14:48 blosc-snappy-5-shuffle-denorm.h5
-rw-r--r-- 1 tomkooij 197613 5.5M Jun 19 14:48 blosc-snappy-5-shuffle.h5
-rw-r--r-- 1 tomkooij 197613 6.1M Jun 19 14:48 blosc-zlib-5-shuffle-denorm.h5
-rw-r--r-- 1 tomkooij 197613 4.4M Jun 19 14:48 blosc-zlib-5-shuffle.h5
-rw-r--r-- 1 tomkooij 197613 5.5M Jun 19 14:48 blosc-zstd-5-shuffle-denorm.h5
-rw-r--r-- 1 tomkooij 197613 4.3M Jun 19 14:48 blosc-zstd-5-shuffle.h5
-rw-r--r-- 1 tomkoo

## Querying in PyTables

Searching in tables is one of the most common and time consuming operations that a typical user faces in the process of mining through his data. Being able to perform queries as fast as possible is a key concept in data usage applications.


In [5]:
# Movieslens-1M (denormalized) not compressed:
fn = "compression/no-compression-denorm.h5"
h5file = tables.open_file(fn)
table = h5file.root.lens

### read_where()

`table.read_where()` reads all table rows that match a query:

In [6]:
table.read_where("rating >= 4")

array([(1, 5, 978824268, b'Toy Story (1995)', b"Animation|Children's|Comedy"),
       (6, 4, 978237008, b'Toy Story (1995)', b"Animation|Children's|Comedy"),
       (8, 4, 978233496, b'Toy Story (1995)', b"Animation|Children's|Comedy"),
       ...,
       (5812, 4, 992072099, b'Contender, The (2000)', b'Drama|Thriller'),
       (5837, 4, 1011902656, b'Contender, The (2000)', b'Drama|Thriller'),
       (5998, 4, 1001781044, b'Contender, The (2000)', b'Drama|Thriller')], 
      dtype=[('user_id', '<i4'), ('rating', 'i1'), ('unix_timestamp', '<i8'), ('title', 'S100'), ('genres', 'S50')])

In [7]:
%%timeit
x = table.read_where("rating >= 4")
max(x['unix_timestamp'])

1.43 s ± 25.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit x = table.read_where("rating >= 4")

1.52 s ± 93.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


###  table.iterrows()

`table.iterrows()` returns an iterator that iterates over ALL rows, using this iterator, we can avoid loading the table in memory.

In [9]:
%%timeit
x = max(x['unix_timestamp'] for x in table.iterrows() if x['rating'] >= 4)

500 ms ± 42.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Better, but we can do much better still:

### table.where()

`table.where()` is an iterator that performs an in-kernel query:

It returns a row iterator, that iterates over the selected rows:

In [10]:
type(table.where('rating >= 4'))

tables.tableextension.Row

In [11]:
%%timeit
ts = (row['unix_timestamp'] for row in table.where("rating >= 4"))
max(ts)

314 ms ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Normalized vs Denormalized tables

Let's compare a "real life" query:

Query the ratings for the movie `Tom and Huck (1995)`:

### Denormalized

In [12]:
h5denorm = "compression/blosc-zstd-5-shuffle-denorm.h5"
h5file = tables.open_file(h5denorm)
h5lens = h5file.root.lens

In [13]:
h5lens

/lens (Table(1000209,), shuffle, blosc:zstd(5)) ''
  description := {
  "user_id": Int32Col(shape=(), dflt=0, pos=0),
  "rating": Int8Col(shape=(), dflt=0, pos=1),
  "unix_timestamp": Int64Col(shape=(), dflt=0, pos=2),
  "title": StringCol(itemsize=100, shape=(), dflt=b'', pos=3),
  "genres": StringCol(itemsize=50, shape=(), dflt=b'', pos=4)}
  byteorder := 'little'
  chunkshape := (402,)

In [14]:
%%timeit
ratings = [0] * 6
for rt in range(0,6):
    ratings[rt] = sum(1 for r in h5lens.where("(title == b'Tom and Huck (1995)') & (rating == rt)"))

2.13 s ± 65.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
ratings

NameError: name 'ratings' is not defined

In [16]:
h5file.close()

Querying denormalized tables is easy as pie.  Let's see how to manage normalized ones.

### Normalized tables

In [17]:
h5norm = "compression/blosc-zstd-5-shuffle.h5"
h5file = tables.open_file(h5norm)
h5ratings = h5file.root.ratings
h5movies = h5file.root.movies

In [18]:
h5ratings

/ratings (Table(1000209,), shuffle, blosc:zstd(5)) ''
  description := {
  "user_id": Int32Col(shape=(), dflt=0, pos=0),
  "movie_id": Int32Col(shape=(), dflt=0, pos=1),
  "rating": Int8Col(shape=(), dflt=0, pos=2),
  "unix_timestamp": Int64Col(shape=(), dflt=0, pos=3)}
  byteorder := 'little'
  chunkshape := (7710,)

In [23]:
h5movies

/movies (Table(3883,), shuffle, blosc:zstd(5)) ''
  description := {
  "movie_id": Int32Col(shape=(), dflt=0, pos=0),
  "title": StringCol(itemsize=100, shape=(), dflt=b'', pos=1),
  "genres": StringCol(itemsize=50, shape=(), dflt=b'', pos=2)}
  byteorder := 'little'
  chunkshape := (425,)

In [26]:
%%time
ratings = [0] * 6
for rt in range(0,6):
    th_movie_id = [r['movie_id'] for r in h5movies.where("(title == b'Tom and Huck (1995)')")][0]
    ratings[rt] = sum(1 for r in h5ratings.where("(movie_id == th_movie_id) & (rating == rt)"))

Wall time: 416 ms


In [27]:
ratings

[0, 4, 15, 28, 18, 3]

In [28]:
h5file.close()

So, the query in the normalized version is more than 2~3x faster than using the denormalized file.  However, this is just a simple example, and in general experimentation should be done so as to determine the best layout for your data.

## Indexing

Indexing is a general technique for adding data structures that can accelerate queries.  Let's see how PyTables makes use of this.

### Denormalized case

In [29]:
## Copy the original PyTables table into another file
import shutil
h5idx = os.path.join(data_dir, "movielens-denorm-indexed.h5")
if os.path.exists(h5idx):
    os.unlink(h5idx)
shutil.copyfile(h5denorm, h5idx)

'queries\\movielens-denorm-indexed.h5'

In [30]:
# Open the new file in 'a'ppend mode
h5i = tables.open_file(h5idx, mode="a")

In [31]:
# Create an index for the 'title' column
h5lens = h5i.root.lens
blosc_filter = tables.Filters(complevel=9, complib="blosc")
%time h5lens.cols.title.create_csindex(filters=blosc_filter)

Wall time: 3.26 s


1000209

In [32]:
%%time
ratings = [0] * 6
for rt in range(0,6):
    ratings[rt] = sum(1 for r in h5lens.where("(title == b'Tom and Huck (1995)') & (rating == rt)"))

Wall time: 11.5 ms


Ok, so this time is 100x less than without using indexing.  What if we index the `rating` column too?

In [33]:
ratings

[0, 4, 15, 28, 18, 3]

In [34]:
# Create an index for the rating column
%time h5lens.cols.rating.create_csindex(filters=blosc_filter)

Wall time: 778 ms


1000209

In [35]:
%%time
ratings = [0] * 6
for rt in range(0,6):
    ratings[rt] = sum(1 for r in h5lens.where("(title == b'Tom and Huck (1995)') & (rating == rt)"))

Wall time: 5 ms


Ok, so although small, this represents another improvement in performance.

In [36]:
ratings

[0, 4, 15, 28, 18, 3]

In [37]:
h5i.close()

### Normalized case

In [109]:
## Copy the original PyTables table into another file
import shutil
h5idx = os.path.join(data_dir, "movielens-norm-indexed.h5")
if os.path.exists(h5idx):
    os.unlink(h5idx)
shutil.copyfile(h5norm, h5idx)

'queries\\movielens-norm-indexed.h5'

In [110]:
# Open the new file in 'a'ppend mode
h5i = tables.open_file(h5idx, mode="a")
h5ratings = h5i.root.ratings
h5movies = h5i.root.movies

In [111]:
# Create an index for the rating column
blosc_filter = tables.Filters(complevel=9, complib="blosc")
%time h5ratings.cols.rating.create_csindex(filters=blosc_filter)

Wall time: 538 ms


1000209

In [112]:
%%time
ratings = [0] * 6
for rt in range(6):
    th_movie_id = [r['movie_id'] for r in h5movies.where("(title == b'Tom and Huck (1995)')")][0]
    ratings[rt] = sum(1 for r in h5ratings.where("(movie_id == th_movie_id) & (rating == rt)"))

Wall time: 426 ms


Hmm, in this case indexing the rating column has not served to accelerate the query (at first sight at least).

In [113]:
ratings

[0, 4, 15, 28, 18, 3]

In [114]:
# Create an index for the movie_id column
%time h5ratings.cols.movie_id.create_csindex(filters=blosc_filter)

Wall time: 618 ms


1000209

In [115]:
%%time
ratings = [0] * 6
for rt in range(6):
    th_movie_id = [r['movie_id'] for r in h5movies.where("(title == b'Tom and Huck (1995)')")][0]
    ratings[rt] = sum(1 for r in h5ratings.where("(movie_id == th_movie_id) & (rating == rt)"))

Wall time: 45 ms


This time we see a better acceleration in the query, but cannot compete with the query speed for the denormalized case (which is ~10x faster).

In [116]:
ratings

[0, 4, 15, 28, 18, 3]

In [117]:
h5i.close()

In [118]:
!ls -lh {data_dir}

total 20M
-rw-r--r-- 1 tomkooij 197613 9.9M Jun 20 09:10 movielens-denorm-indexed.h5
-rw-r--r-- 1 tomkooij 197613 9.6M Jun 20 09:10 movielens-norm-indexed.h5


## Exercise

We have not created an index for the title for the normalized case.  Create such an index and determine if there is a noticeable speed-up or not.  Explain why you think that is the case.  Note: the times for a cold query can be **significatively** different from a hot query.

In [119]:
## Copy the original PyTables table into another file
import shutil
h5idx2 = "movielens-norm-indexed2.h5"
if os.path.exists(h5idx2):
    os.unlink(h5idx2)
shutil.copyfile(h5idx, h5idx2)

'movielens-norm-indexed2.h5'

In [1]:
# Open the new file in 'a'ppend mode
h5i = tables.open_file(h5idx2, mode="a")
h5ratings = h5i.root.ratings
h5movies = h5i.root.movies

NameError: name 'tables' is not defined

In [121]:
#
#
# Solution starts here
#
#

In [122]:
# Create an index for the movie_id column
%time h5movies.cols.title.create_csindex(filters=blosc_filter)

Wall time: 17 ms


3883

In [123]:
%%time
ratings = [0] * 6
for rt in range(6):
    th_movie_id = [r['movie_id'] for r in h5movies.where("(title == b'Tom and Huck (1995)')")][0]
    ratings[rt] = sum(1 for r in h5ratings.where("(movie_id == th_movie_id) & (rating == rt)"))

Wall time: 253 ms


In [124]:
ratings

[0, 4, 15, 28, 18, 3]

In [125]:
h5i.close()

So the first time that the query is done after the cache is built (cold query), the time has been reduced a bit but not too much.  For subsequent queries (hot queries), the times are better, but not reaching the denormalized table either.

### Exercise

Query size vs speed (indexed queries vs non-indexed queries)

In [7]:
# adapted from: https://stackoverflow.com/questions/20769818/

import random
import string

class KeyValue(tables.IsDescription):
    key = tables.StringCol(itemsize=30, dflt=" ", pos=0)  
    value = tables.Int64Col(dflt=0, pos=1)

fn = os.path.join(data_dir, "keyvalue.h5")

with tables.open_file(fn, "w") as f:    
    filters = tables.Filters(complevel=5, complib='blosc')
    kv = f.create_table("/", "keyvalues", KeyValue, filters=filters)

    for j in range(200):
        values = []
        print('iteration: ', j)
        for _ in range(100000):
            key = "".join(random.sample(string.ascii_uppercase, 10))  # slow!
            value = random.randint(0, 1000000)
            values.append((key, value))
        kv.append(values)

iteration:  0
iteration:  1
iteration:  2
iteration:  3
iteration:  4
iteration:  5
iteration:  6
iteration:  7
iteration:  8
iteration:  9
iteration:  10
iteration:  11
iteration:  12
iteration:  13
iteration:  14
iteration:  15
iteration:  16
iteration:  17
iteration:  18
iteration:  19
iteration:  20
iteration:  21
iteration:  22
iteration:  23
iteration:  24
iteration:  25
iteration:  26
iteration:  27
iteration:  28
iteration:  29
iteration:  30
iteration:  31
iteration:  32
iteration:  33
iteration:  34
iteration:  35
iteration:  36
iteration:  37
iteration:  38
iteration:  39
iteration:  40
iteration:  41
iteration:  42
iteration:  43
iteration:  44
iteration:  45
iteration:  46
iteration:  47
iteration:  48
iteration:  49
iteration:  50
iteration:  51
iteration:  52
iteration:  53
iteration:  54
iteration:  55
iteration:  56
iteration:  57
iteration:  58
iteration:  59
iteration:  60
iteration:  61
iteration:  62
iteration:  63
iteration:  64
iteration:  65
iteration:  66
itera

In [8]:
!ptdump -v -R10 {fn}

/ (RootGroup) ''
/keyvalues (Table(20000000,), shuffle, blosc(5)) ''
  description := {
  "key": StringCol(itemsize=30, shape=(), dflt=b' ', pos=0),
  "value": Int64Col(shape=(), dflt=0, pos=1)}
  byteorder := 'little'
  chunkshape := (1724,)
  Data dump:
[0] (b'QPGONRWHJX', 651797)
[1] (b'HNGTUJYKAP', 313014)
[2] (b'NRALQITYZV', 225736)
[3] (b'YPLNGDVTHK', 210883)
[4] (b'BXGIQLMFUE', 482386)
[5] (b'OUSZQHPGFJ', 52527)
[6] (b'GTZVRCMYNJ', 107035)
[7] (b'CVAIYMLQHU', 92337)
[8] (b'RYZCGWBTXL', 143533)
[9] (b'HGOUALQSIC', 888223)


In [6]:
max_values = [10, 50, 100, 1000, 10000]

with tables.open_file(fn, "a") as f:
    kv = f.root.keyvalues

    kv.cols.value.remove_index()

    for max_value in max_values:
        print('max_value=%d : len=%d' % (max_value, len(kv.read_where('value < %s' % max_value))))
    
    print('\nwithout index:')
    for max_value in max_values:
        #%time kv.read_where('value < %s' % max_value)
        %time sum(1 for x in kv.where('value < %s' % max_value))

    blosc_filter = tables.Filters(complevel=9, complib="blosc")
    print('\nindexing...')
    %time kv.cols.value.create_csindex()

    print('\nwith index')
    for max_value in max_values:
        #%time kv.read_where('value < %s' % max_value)
        %time sum(1 for x in kv.where('value < %s' % max_value))

max_value=10 : len=18
max_value=50 : len=52
max_value=100 : len=99
max_value=1000 : len=971
max_value=10000 : len=10031

without index:
Wall time: 76.1 ms
Wall time: 82.6 ms
Wall time: 72.6 ms
Wall time: 85 ms
Wall time: 74.5 ms

indexing...
Wall time: 1.66 s

with index
Wall time: 8.01 ms
Wall time: 8.52 ms
Wall time: 18 ms
Wall time: 73.6 ms
Wall time: 92.6 ms


In [51]:
f = tables.open_file(fn, 'r')
f
f.close()

File(filename=queries\keyvalue.h5, title='', mode='r', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) ''
/keyvalues (Table(10000000,)) ''
  description := {
  "key": StringCol(itemsize=30, shape=(), dflt=b' ', pos=0),
  "value": Int64Col(shape=(), dflt=0, pos=1)}
  byteorder := 'little'
  chunkshape := (1724,)
  autoindex := True
  colindexes := {
    "value": Index(9, full, shuffle, zlib(1)).is_csi=True}